In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import re
from itertools import chain

In [3]:
import z3

In [ ]:
x = z3.And(z3.Bool("a"),z3.Bool("b")); print(x)

In [ ]:
?z3.z3printer.Formatter

In [ ]:
formatter = z3.z3printer.Formatter()

In [ ]:
formatter.pp_bv(x)

In [4]:
def collapse_whitespace(s):
    rex = re.compile(r'\s+')
    return rex.sub(' ', s)

In [5]:
s = """And(And(Not(passenger-in-taxi(curly, t0)),
        Not(passenger-in-taxi(smoov, t0)),
        Not(passenger-in-taxi(littman, t0)),
        Not(passenger-in-taxi(isbell, t0))))"""
s = collapse_whitespace(s)

In [6]:
def get_matching_parens(s):
    paren_pairs = []
    left_parens_stack = []
    for i, c in enumerate(s):
        if c == "(":
            left_parens_stack.append(i)
        if c == ")":
            if len(left_parens_stack) == 0:
                raise ValueError(f"Unbalanced Parens: {i}")
            paren_pairs.append((left_parens_stack.pop(), i))
    if len(left_parens_stack) > 0:
        raise ValueError(f"Extra left parens: {left_parens_stack}")
    return sorted(paren_pairs, key=lambda x: x[0])

In [8]:
def overlapping(a,b):
    # Make sure a comes first
    if a[0] > b[0]:
        a_ = a
        a = b
        b = a_
        del a_
    if a[1] >= b[0]:
        return True
    else:
        return False

In [9]:
def range_contain(a,b):
    return a[0] <= b[0] and a[1] >= b[1]

In [10]:
def get_maximal_ranges(ranges):
    ranges = sorted(ranges, key=lambda x: x[0])
    maximals = []
    for r in ranges:
        add_r = True
        for m in maximals:
            if range_contain(m,r):
                add_r = False
                continue
            elif range_contain(r,m):
                maximals.pop(m)
        if add_r:
            maximals.append(r)
    return maximals

In [12]:
paren_pairs = get_matching_parens(s); paren_pairs

[(3, 150),
 (7, 149),
 (11, 40),
 (29, 39),
 (46, 75),
 (64, 74),
 (81, 112),
 (99, 111),
 (118, 148),
 (136, 147)]

In [13]:
maximal_paren_pairs = get_maximal_ranges(paren_pairs); maximal_paren_pairs

[(3, 150)]

In [15]:
def index_complement(s, subtracted_ranges):
    # Assumes that subtracted_ranges are non overlapping
    subtracted_ranges = sorted(subtracted_ranges, key=lambda x: x[0])
    new_ranges = []
    start = 0
    for r in subtracted_ranges:
        if r[0] > start:
            new_ranges.append((start, r[0]))
        start = r[1] + 1
    if start < len(s):
        new_ranges.append((subtracted_ranges[-1][1], len(s)))
    return new_ranges

In [16]:
index_complement(s, maximal_paren_pairs)

[(0, 3)]

In [19]:
maximal_paren_pairs[0]

(3, 150)

In [23]:
s[maximal_paren_pairs[0][0] + 1:maximal_paren_pairs[0][1]]

'And(Not(passenger-in-taxi(curly, t0)), Not(passenger-in-taxi(smoov, t0)), Not(passenger-in-taxi(littman, t0)), Not(passenger-in-taxi(isbell, t0)))'

In [27]:
def parse_operators(s):
    maximal_paren_pairs = get_maximal_ranges(get_matching_parens(s))
    str_ranges = index_complement(s, maximal_paren_pairs)
    assert len(maximal_paren_pairs) == len(str_ranges)
    ret = []
    for i in range(len(maximal_paren_pairs)):
        op_range = str_ranges[i]
        op = s[op_range[0]:op_range[1]]
        arg_range = maximal_paren_pairs[i]
        arg_s = s[arg_range[0] + 1:arg_range[1]]
        arg = parse_operators(arg_s)
        ret.append((op, arg))
    return ret

In [28]:
parse_operators(s)

IndexError: list index out of range

In [ ]:
# stack = []
# # l = []
# s_ = ""
# for c in s:
#     # Go in one level
#     if c == "(":
#         stack.append(l)
#         l = []
#     # Go up a level
#     elif c == ")":
#         pass
#     else:
#         s_ += c

In [ ]:
left_parens = [x.start() for x in re.finditer("\(",s)]
right_parens = [x.start() for x in re.finditer("\)",s)]

In [ ]:
assert len(left_parens) == len(right_parens)


In [ ]:
print([s[i] for i in left_parens])

In [ ]:
dum = [1,2,3]
dum.pop()
print(dum)

In [ ]:
?sorted

In [ ]:
s = collapse_whitespace(s); print(s)

In [ ]:
paren_pairs = get_matching_parens(s)

In [ ]:
split_idx = sorted(list(chain(*paren_pairs)))
if 0 not in split_idx:
    split_idx = [-1] + split_idx

In [ ]:
s_pieces = []
for i in range(len(split_idx) - 1):
    s_pieces.append(s[split_idx[i] + 1:split_idx[i+1]])

In [ ]:
s_pieces


In [ ]:
s

In [ ]:
def parse2(my_str):
    stack = []
    l = []
    for t in re.findall(r'[()]|[^\s()]+', my_str):
        print(t)
        if t == '(':
            stack.append(l)
            l = []
        elif t == ')':
            if stack:
                l2 = l
                l = stack.pop()
                l.append(l2)
            else:
                raise Exception('Missing open parentheses')
        else:
            l.append(t)
    if stack:
        raise Exception('Missing close parentheses')
    # print(l)
    # if len(l) != 1:
        # raise Exception('Malformed expression')
    # return l[0]
    return l

In [ ]:
l = parse2(s); l

In [ ]:
nested_list2str(l)

In [ ]:
s

In [ ]:
def listify(s, paren_pairs):
    

In [ ]:
# https://stackoverflow.com/a/23185606
def parse_nested(text, left=r'[(]', right=r'[)]', sep=r','):
    """ Based on https://stackoverflow.com/a/17141899/190597 (falsetru) """
    pat = r'({}|{}|{})'.format(left, right, sep)
    tokens = re.split(pat, text)    
    stack = [[]]
    for x in tokens:
        if not x or re.match(sep, x): continue
        if re.match(left, x):
            stack[-1].append([])
            stack.append(stack[-1][-1])
        elif re.match(right, x):
            stack.pop()
            if not stack:
                raise ValueError('error: opening bracket is missing')
        else:
            stack[-1].append(x)
    if len(stack) > 1:
        print(stack)
        raise ValueError('error: closing bracket is missing')
    return stack.pop()

In [ ]:
l = parse_nested(s); l

In [ ]:
len(l)

In [ ]:
print("~")

In [ ]:
operator_symbols = {
    "And": "&",
    "Or": "|",
}
def nested_list2str(l):
    if isinstance(l, str):
        return l
    # elif list_is_flat(l):
    #     return ",".join(l)
    else:
        assert len(l) == 2, f"{len(l)}; {l}"
        operator, args = l
        args = [nested_list2str(x) for x in args]
        if operator == "Not":
            assert len(args) == 1, l
            return f"~{args[0]}"
        else:
            operator_symbol = operator_symbols[operator]
            return "(" + operator_symbol.join(args)



        

In [ ]:
nested_list2str(l)

In [ ]:
l[0]

In [ ]:
l[1]

In [ ]:
l

In [ ]:
len(l)

In [ ]:
len(l[1][0])